In [ ]:
import os
class EXTRATOR_CNPJ: 
    def __init__(self, url:str = None,nome_arquivo:str = '', extensao:str='zip'): 
        self.url = url 
        self.nome_arquivo = nome_arquivo
        self.extensao = extensao 
        self.CNPJ_ESQUEMA =  {
                        "EMPRESAS" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "RAZAO_SOCIAL",
                                "_c2" : "CODIGO_NATUREZA_JURIDICA",
                                "_c3" : "CODIGO_QUALIFICACAO",
                                "_c4" : "CAPITAL_SOCIAL",
                                "_c5" : "PORTE_EMPRESA",
                                "_c6" : "ENTE_FEDERATIVO"
                                },
                        "ESTABELECIMENTOS" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "CNPJ_ORDEM",
                                "_c2" : "CNPJ_DV",
                                "_c3" : "MATRIZ_FILIAL",
                                "_c4" : "NOME_FANTASIA",
                                "_c5" : "SITUACAO_CADASTRAL",
                                "_c6" : "DATA_SITUACAO_CADASTRAL",
                                "_c7" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL",
                                "_c8" : "CIDADE_EXTERIOR",
                                "_c9" : "CODIGO_PAIS",
                                "_c10" : "DATA_INICIO_ATIVIDADE",
                                "_c11" : "CNAE_PRINCIPAL",
                                "_c12" : "CNAE_SECUNDARIO",
                                "_c13" : "TIPO_LOGRADOURO",
                                "_c14" : "LOGRADOURO",
                                "_c15" : "NUMERO",
                                "_c16" : "COMPLEMENTO",
                                "_c17" : "BAIRRO",
                                "_c18" : "CEP",
                                "_c19" : "UF",
                                "_c20" : "CODIGO_MUNICIPIO",
                                "_c21" : "DDD_CONTATO",
                                "_c22" : "TELEFONE_CONTATO",
                                "_c23" : "DDD_COMERCIAL",
                                "_c24" : "TELEFONE_COMERCIAL",
                                "_c25" : "DDD_FAX",
                                "_c26" : "FAX",
                                "_c27" : "EMAIL",
                                "_c28" : "SITUACAO_ESPECIAL",
                                "_c29" : "DATA_SITUACAO_ESPECIAL"
                                },
                        "SIMPLES" : {
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "OPCAO_SIMPLES",
                                "_c2" : "DATA_OPCAO_SIMPLES",
                                "_c3" : "DATA_EXCLUSAO_SIMPLES",
                                "_c4" : "OPCAO_MEI",
                                "_c5" : "DATA_OPCAO_MEI",
                                "_c6" : "DATA_EXCLUSAO_MEI"
                                },
                        "SOCIOS" :{
                                "_c0" : "CNPJ_BASE",
                                "_c1" : "TIPO_SOCIO",
                                "_c2" : "NOME_SOCIO",
                                "_c3" : "CPF_CNPJ",
                                "_c4" : "CODIGO_QUALIFICACAO_SOCIO",
                                "_c5" : "DATA_ENTRADA_SOCIEDADE",
                                "_c6" : "PERCENTUAL_CAPITAL_SOCIAL",
                                "_c7" : "CPF_REPRESENTANTE",
                                "_c8" : "NOME_REPRESENTANTE",
                                "_c9" : "CODIGO_QUALIFICACAO_REPRESENTANTE",
                                "_c10" : "FAIXA_ETARIA"
                                },
                        "PAISES" : {
                                "_c0" : "CODIGO_PAIS", 
                                "_c1" :"NOME_PAIS"
                                },
                        "MUNICIPIOS" : {
                                "_c0" : "CODIGO_MUNICIPIO", 
                                "_c1" : "NOME_MUNICIPIO"
                                },
                        "QUALIFICACOES" : {
                                "_c0" :"CODIGO_QUALIFICACAO", 
                                "_c1" : "DESCRIÇAO_QUALIFICACAO"
                                },
                        "NATUREZAS" : {
                                "_c0" :"CODIGO_NATUREZA_JURIDICA", 
                                "_c1" : "DESCRICAO_NATUREZA_JURIDICA"
                                },
                        "MOTIVOS" : {
                                "_c0" : "CODIGO_MOTIVO_SITUACAO_CADASTRAL", 
                                "_c1" :"DESCRICAO_MOTIVO_SITUACAO_CADASTRAL"
                                },
                        "CNAES" : {
                                "_c0" : "CODIGO_CNAE",
                                "_c1" : "DESCRICAO_CNAE"
                                } 
                        }
    def request(self):
        import re
        import urllib.request
        abre_url = urllib.request.urlopen(self.url)
        le_url = abre_url.read()
        converte_leitura_url = le_url.decode("utf8")
        abre_url.close()
        urls = []
        urls =  [ f"{self.url}{i.upper()}.{self.extensao}" for i in re.findall(f'href="(.*).(?:{self.extensao})"', converte_leitura_url) if i.startswith(self.nome_arquivo)]
        return urls

    def download(self, url:str=None, destino:str = None):
        ### Com requests
        """
        import requests
        with requests.get(url, stream=True) as response:
            with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)

        """
        ### Com urllib
        """
        import urllib.request
        arquivo = url.split('/')[-1]
        # faz o download do arquivo e salva em salvar_onde/arquivo
        urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))
        ### Com SmartDL
        """

        ### Com SmartDL
        from pySmartDL import SmartDL

        obj = SmartDL(url, destino, threads=4, progress_bar=False)
        obj.start()
        return destino

    def run(self):
        import re
        import zipfile
        from pyspark.sql import SparkSession
        from pyspark.sql.functions import input_file_name, lit



        # Define ou busca uma sessão do Spark
        spark = (
            SparkSession.builder.master("local[2]")
            .appName("OnlineReader")
            .getOrCreate()
        )
        spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "true")
        
        # ativa o método request
        urls = self.request()
        for url in urls:
            # numero_arquivo = urls.index(url)
            # Diretório de execução do script
            current_dir = os.path.dirname(os.path.abspath(__file__))

            # Pega o nome do arquivo pela url
            nome_arquivo_download = url.split("/")[-1]

            # Define o caminho absoluto do diretório.
            salvar_onde = f"{current_dir}/RAW/{''.join(filter(str.isalpha,nome_arquivo_download.split('.')[0]))}/"
            # cria a pasta para armazenar o arquivo, se ela não existir
            if not os.path.exists(salvar_onde):
                os.makedirs(salvar_onde)

            # download do arquivo zip
            path = self.download(url, os.path.join(salvar_onde, nome_arquivo_download))

            # descompactação do arquivo zip
            with zipfile.ZipFile(path, "r") as zip_ref:
                # obtem o nome do primeiro arquivo dentro do zip
                nome_original_arquivo_zip = zip_ref.namelist()[0]

                # define um novo nome para o arquivo
                novo_nome_arquivo = f"CNPJ_{nome_arquivo_download.split('.')[0]}.csv"
                # cria um dicionário com as informações de origem e destino
                arquivos_para_extrair = {nome_original_arquivo_zip: novo_nome_arquivo}

                # realiza a extração do arquivo zip
                zip_ref.extractall(path=f"{salvar_onde}", members=arquivos_para_extrair)

                # renomeia o arquivo extraído com o novo nome
                os.replace(
                    os.path.join(salvar_onde, nome_original_arquivo_zip),
                    os.path.join(salvar_onde, novo_nome_arquivo),
                )
            os.remove(path)
        # leitura do arquivo CSV em um dataframe Spark
        # Obtém a lista de arquivos no diretório
        lista_arquivos_no_diretorio = [
            os.path.join(salvar_onde, nome)
            for nome in os.listdir(salvar_onde)
            if nome.endswith(".csv")
        ]
        # Lê o arquivo CSV em um dataframe Spark e adiciona uma coluna com o nome do arquivo
        if len(lista_arquivos_no_diretorio) == 1:
            dados = (
                spark.read.format("csv")
                .option("header", "false")
                .option("delimiter", ";")
                .option("inferSchema", "true")
                .load(lista_arquivos_no_diretorio[0])
            )
            dados.withColumn("NOME_ARQUIVO", lit(lista_arquivos_no_diretorio[0].split("\\")[-1]))
        # Lê cada arquivo CSV em um dataframe Spark e adiciona uma coluna com o nome do arquivo
        else:
            dados = None
            for arquivo_no_diretorio in lista_arquivos_no_diretorio:
                if dados is None:
                    dados = (
                        spark.read.format("csv")
                        .option("header", "false")
                        .option("delimiter", ";")
                        .option("inferSchema", "true")
                        .load(arquivo_no_diretorio)
                    )
                    dados = dados.withColumn(
                        "NOME_ARQUIVO", lit(arquivo_no_diretorio.split("\\")[-1])
                    )
                else:
                    # print(arquivo_no_diretorio)
                    dados_incrementados = (
                        spark.read.format("csv")
                        .option("header", "false")
                        .option("delimiter", ";")
                        .option("inferSchema", "true")
                        .load(arquivo_no_diretorio)
                    )
                    dados_incrementados = dados_incrementados.withColumn(
                        "NOME_ARQUIVO", lit(arquivo_no_diretorio.split("\\")[-1])
                    )
                    dados = dados.union(dados_incrementados)
        # USA O MÉTODO WITHCOLUMNRENAMED() PARA RENOMEAR AS COLUNAS
        for NOME_ANTIGO, NOVO_NOME in self.CNPJ_ESQUEMA[
            "".join(filter(str.isalpha, nome_arquivo_download.split(".")[0]))
        ].items():
            dados = dados.withColumnRenamed(NOME_ANTIGO, NOVO_NOME)
        return dados


In [ ]:
url = 'https://dadosabertos.rfb.gov.br/CNPJ/'
EC_SOCIOS = EXTRATOR_CNPJ(url=url, nome_arquivo='Socios', extensao='zip')
SOCIOS = EC_SOCIOS.run()

In [1]:
import os
from DOWNLOAD_EXTRACT_TO_TRANSFORM_CNPJ import EXTRATOR_CNPJ
current_dir = os.getcwd()
dir_dados = os.path.join(current_dir, r"dados")

In [2]:
# Se passar baixar_e_extrair como false, precisa do nome do arquivo.
ESTABELECIMENTOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Estabelecimentos").run()
print(f"Este dataframe tem {ESTABELECIMENTOS.count()} Observações")

-RECORD 0------------------------------------------------
 CNPJ_BASE                        | 27459772             
 CNPJ_ORDEM                       | 1                    
 CNPJ_DV                          | 42                   
 MATRIZ_FILIAL                    | 1                    
 NOME_FANTASIA                    | J M NETO             
 SITUACAO_CADASTRAL               | 8                    
 DATA_SITUACAO_CADASTRAL          | 20180821             
 CODIGO_MOTIVO_SITUACAO_CADASTRAL | 1                    
 CIDADE_EXTERIOR                  | null                 
 CODIGO_PAIS                      | null                 
 DATA_INICIO_ATIVIDADE            | 20170404             
 CNAE_PRINCIPAL                   | 4754702              
 CNAE_SECUNDARIO                  | null                 
 TIPO_LOGRADOURO                  | RUA                  
 LOGRADOURO                       | B                    
 NUMERO                           | 120                  
 COMPLEMENTO  

In [3]:
EMPRESAS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Empresas").run()
print(f"Este dataframe tem {EMPRESAS.count()} Observações")

-RECORD 0----------------------------------------
 CNPJ_BASE                | 41273597             
 RAZAO_SOCIAL             | PACHARRUS QUEIROZ... 
 CODIGO_NATUREZA_JURIDICA | 2135                 
 CODIGO_QUALIFICACAO      | 50                   
 CAPITAL_SOCIAL           | 5000,00              
 PORTE_EMPRESA            | 1                    
 ENTE_FEDERATIVO          | null                 
 NOME_ARQUIVO             | rfb/RAW/Empresas/... 
only showing top 1 row

Este dataframe tem 53293844 Observações


In [4]:
MUNICIPIOS, spark = EXTRATOR_CNPJ(baixar_e_extrair=False, nome_arquivo="Municipios").run()
print(f"Este dataframe tem {MUNICIPIOS.count()} Observações")

-RECORD 0-------------------------
 CODIGO_MUNICIPIO | 1             
 NOME_MUNICIPIO   | GUAJARA-MIRIM 
only showing top 1 row

Este dataframe tem 5571 Observações


In [5]:
# Cria uma view com o mesmo nome do DataFrame
ESTABELECIMENTOS.createOrReplaceTempView("ESTABELECIMENTOS")
EMPRESAS.createOrReplaceTempView("EMPRESAS")
MUNICIPIOS.createOrReplaceTempView("MUNICIPIOS")

CNAES = {
        5612100:'Serviços ambulantes de alimentação',
        5611201:'Restaurantes e similares',
        5611203:'Lanchonetes casas de chá de sucos e similares',
        5611204:'Bares e outros estabelecimentos especializados em servir bebidas sem entretenimento',
        5611205:'Bares e outros estabelecimentos especializados em servir bebidas com entretenimento',
        4721102: 'Padaria e confeitaria com predominância de revenda'
        }
# Consulta a view em SQL

In [ ]:
spark.sql("""SELECT t1.CNAE_PRINCIPAL, t1.QUANTIDADE_FECHADA, t2.QUANTIDADE_GERAL, t3.QUANTIDADE_ABERTA
            FROM (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_FECHADA
            FROM dados
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205') AND SITUACAO_CADASTRAL NOT IN (2, 3, 4)
            GROUP BY CNAE_PRINCIPAL
            ) AS t1
            FULL OUTER JOIN (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_GERAL
            FROM dados 
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205')
            GROUP BY CNAE_PRINCIPAL 
            ) AS t2 
            ON t1.CNAE_PRINCIPAL = t2.CNAE_PRINCIPAL
            FULL OUTER JOIN (
            SELECT CNAE_PRINCIPAL, COUNT(*) AS QUANTIDADE_ABERTA
            FROM dados
            WHERE CNAE_PRINCIPAL IN ('4721102','5612100', '5611201', '5611203', '5611204', '5611205') AND SITUACAO_CADASTRAL IN (2, 3, 4)
            GROUP BY CNAE_PRINCIPAL
            ) AS t3 
            ON t1.CNAE_PRINCIPAL = t3.CNAE_PRINCIPAL
            ORDER BY t1.CNAE_PRINCIPAL
            """).show()

In [8]:
# cria um dataframe com base na query
CNAE_4721102 = spark.sql("""SELECT CONCAT(LPAD(EST.CNPJ_BASE, 8, '0'), LPAD(EST.CNPJ_ORDEM, 4, '0'), LPAD(EST.CNPJ_DV, 2, '0')) AS CNPJ,EMP.RAZAO_SOCIAL, EST.NOME_FANTASIA,
                            EST.SITUACAO_CADASTRAL, EST.DATA_SITUACAO_CADASTRAL, 
                            EST.DATA_INICIO_ATIVIDADE, EST.CNAE_PRINCIPAL, CONCAT(EST.TIPO_LOGRADOURO, EST.LOGRADOURO, EST.NUMERO, EST.COMPLEMENTO) AS ENDERECO, EST.BAIRRO, MUN.NOME_MUNICIPIO AS CIDADE, EST.UF, EST.CEP,
                            CONCAT(EST.DDD_CONTATO, " ", EST.TELEFONE_CONTATO) AS TELEFONE, EST.EMAIL 
                            FROM ESTABELECIMENTOS AS EST
                            RIGHT JOIN EMPRESAS AS EMP ON EST.CNPJ_BASE = EMP.CNPJ_BASE
                            RIGHT JOIN MUNICIPIOS AS MUN ON EST.CODIGO_MUNICIPIO = MUN.CODIGO_MUNICIPIO
                            WHERE EST.CNAE_PRINCIPAL = 4721102 AND EST.SITUACAO_CADASTRAL IN (2, 3, 4)
""")
# salva o DataFrame resultante em um arquivo Parquet
arquivo = os.path.join(dir_dados, "Padaria e confeitaria com predominância de revenda.parquet")
CNAE_4721102.write.mode("overwrite").parquet(arquivo)

### Salva no disco e lê após isso
``` PYTHON
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)
```

In [ ]:
# Libs
import requests # SE FOR USAR REQUESTS
import urllib.request # SE FOR USAR URLLIB
from pySmartDL import SmartDL # SE FOR USAR PYSMARTDL
import zipfile
import os
from pyspark.sql import SparkSession
current_dir = os.getcwd()

# Define ou busca uma sessão do Spark
spark = SparkSession.builder.master("local[2]") \
    .appName("OnlineReader") \
    .getOrCreate()

# Define a url de download dos dados
url = 'https://dadosabertos.rfb.gov.br/CNPJ/Simples.zip'
# Pega o nome do arquivo pela url
arquivo = url.split('/')[-1]
# Define o caminho absoluto do diretório.
salvar_onde = f"{current_dir}/RAW/{arquivo.split('.')[0]}/"

# cria a pasta para armazenar o arquivo, se ela não existir
if not os.path.exists(salvar_onde):
    os.makedirs(salvar_onde)

### Com requests
"""
    with requests.get(url, stream=True) as response:
        with open(os.path.join(salvar_onde, arquivo), 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)

"""
### Com urllib

# faz o download do arquivo e salva em salvar_onde/arquivo
urllib.request.urlretrieve(url, os.path.join(salvar_onde, arquivo))

### Com SmartDL
"""
    dest = os.path.join(salvar_onde, arquivo)
    obj = SmartDL(url, dest, threads=4)
    obj.start()
"""

"""
# Imprime o caminho do diretório de download
print(salvar_onde)
"""

# Descompactação do arquivo
with zipfile.ZipFile(os.path.join(salvar_onde, arquivo), 'r') as zip_ref:

    # obtem o nome do primeiro arquivo dentro do zip
    nome_original_arquivo_zip = zip_ref.namelist()[0]

    # define um novo nome para o arquivo
    novo_nome_arquivo = f"CNPJ_{arquivo.split('.')[0]}.csv"

    # cria um dicionário com as informações de origem e destino
    arquivos_para_extrair = {nome_original_arquivo_zip : novo_nome_arquivo}

    # realiza a extração do arquivo zip
    zip_ref.extractall(path = f"{salvar_onde}/", members=arquivos_para_extrair)
    
    # renomeia o arquivo extraído com o novo nome
    os.rename(os.path.join(salvar_onde, nome_original_arquivo_zip), os.path.join(salvar_onde, novo_nome_arquivo))

"""
# imprime o nome do arquivo dentro do zip e o novo nome
print(f"Arquivo dentro do zip: {nome_original_arquivo_zip}")
print(f"Novo nome do arquivo: {novo_nome_arquivo}")
"""

# Define o caminho absoluto para o arquivo
csv_file = os.path.join(salvar_onde, novo_nome_arquivo)

# Lê o arquivo em um dataframe Spark
dados = spark.read.options(delimiter=";", header=False, inferSchema=True).csv(csv_file)

# Plota primeira linha do dataframe
dados.show(1, vertical=True)

In [ ]:
dados.count()